# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы изучите методы работы с категориальными переменными

In [91]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(2 балла)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [92]:
dataframe = pd.read_csv('train.csv', index_col=0)
labels = dataframe.target.values
dataframe = dataframe.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [93]:
np.random.seed(910)
positive_samples = np.random.choice(np.where(labels == 1)[0], 100000, replace=True)
negative_samples = np.random.choice(np.where(labels == 0)[0], 100000, replace=True)

balanced_data = pd.concat((dataframe.iloc[positive_samples], dataframe.iloc[negative_samples]))
balanced_labels = np.hstack((labels[positive_samples], labels[negative_samples]))

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [94]:
X_train, X_test, y_train, y_test = train_test_split(balanced_data, balanced_labels, test_size=0.5)

def standardize(data):
    scaler = StandardScaler()
    normalized_data = scaler.fit_transform(data)
    return pd.DataFrame(normalized_data, columns=data.columns)

normalized_data = standardize(balanced_data)

cat_columns = [col for col in balanced_data.columns if col.endswith('_cat')]

encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_matrix = encoder.fit_transform(balanced_data[cat_columns])

clean_data = balanced_data.drop(cat_columns, axis=1)
encoded_dataframe = pd.DataFrame(encoded_matrix, columns=encoder.get_feature_names_out(cat_columns))

clean_data.reset_index(inplace=True, drop=True)
final_data = pd.concat([clean_data, encoded_dataframe], axis=1)


Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию, найдтие оптимум. Сделайте предсказание на тестовой части выборки. Замерьте качество.

In [95]:
def evaluate_classifier(train_features, test_features, train_labels, test_labels):
    classifier = LogisticRegression(max_iter=1000)
    classifier.fit(train_features, train_labels)
    predictions = classifier.predict(test_features)

    acc = accuracy_score(test_labels, predictions)
    prec = precision_score(test_labels, predictions)
    rec = recall_score(test_labels, predictions)
    f1_metric = f1_score(test_labels, predictions)

    print(f"Accuracy of the model:", acc)
    print(f"Precision of the model:", prec)
    print(f"Recall of the model:", rec)
    print(f"F1 score of the model:", f1_metric)

In [96]:
%%time
evaluate_classifier(X_train, X_test, y_train, y_test)


Accuracy of the model: 0.59184
Precision of the model: 0.5998965026521196
Recall of the model: 0.555528932550617
F1 score of the model: 0.5768608749740826
CPU times: user 32 s, sys: 9.91 s, total: 41.9 s
Wall time: 34.7 s


__Выводы__ в свободной форме: Точность модели составила 60%. Хотя это не идеально, для некоторых задач бинарной классификации такой результат может быть приемлемым, особенно при наличии сбалансированных классов.



## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [97]:
cat_features = [feature for feature in combined_data.columns if feature.endswith('_cat')]

encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_values = encoder.fit_transform(combined_data[cat_features])

processed_data = combined_data.drop(cat_features, axis=1)
encoded_df = pd.DataFrame(encoded_values, columns=encoder.get_feature_names_out(cat_features))

processed_data.reset_index(drop=True, inplace=True)
processed_data = pd.concat([processed_data, encoded_df], axis=1)

In [98]:
%%time
evaluate_classifier(*train_test_split(standardize(processed_data), combined_target, test_size=0.5, random_state=123))

Accuracy of the model: 0.60239
Precision of the model: 0.6100223428024258
Recall of the model: 0.572192726837252
F1 score of the model: 0.5905022812239307
CPU times: user 5.46 s, sys: 958 ms, total: 6.42 s
Wall time: 4 s


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [99]:
import pandas as pd
import numpy as np
import warnings

counter_data = data.copy()
warnings.simplefilter("ignore")

cat_columns = [col for col in data.columns if col.endswith('cat')]

for column in cat_columns:
    unique_values = np.unique(data[column])
    for value in unique_values:
        target_mean = np.mean(target[data[column] == value])
        counter_data[column][data[column] == value] = target_mean
counter_data.describe()


,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.995400,0.500000,4.485440,0.500000,0.500000,0.350675,0.293855,0.177420,0.178050,0.000530,...,5.447150,1.430895,2.875485,7.546010,0.120480,0.631650,0.559190,0.288245,0.348590,0.150910
std,2.014924,0.012169,2.732782,0.023243,0.052420,0.477183,0.455527,0.382025,0.382556,0.023016,...,2.321879,1.195007,1.701089,2.763712,0.325523,0.482358,0.496485,0.452947,0.476525,0.357962
min,0.000000,0.493976,0.000000,0.482224,0.479814,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.497261,2.000000,0.482224,0.479814,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.497261,4.000000,0.482224,0.479814,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.497261,7.000000,0.522642,0.479814,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.847561,11.000000,0.932886,0.704973,1.000000,1.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,12.000000,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


__Вывод:__

Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

In [100]:
categories = [v for v in data if v.endswith("cat")]

for v in categories:
    unique_values = np.unique(data[v])
    for value in unique_values:
        target_mean = np.mean(target[data[v] == value])
        counter_data[v][data[v] == value] = target_mean

for v in categories:
    value_counts = data[v].value_counts()
    counter_data[v] = data[v].map(value_counts)

counter_data.describe()


,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.995400,113923.514270,4.485440,101707.709010,147229.412560,0.350675,0.293855,0.177420,0.178050,0.000530,...,5.447150,1.430895,2.875485,7.546010,0.120480,0.631650,0.559190,0.288245,0.348590,0.150910
std,2.014924,50528.143146,2.732782,13775.244222,58023.781212,0.477183,0.455527,0.382025,0.382556,0.023016,...,2.321879,1.195007,1.701089,2.763712,0.325523,0.482358,0.496485,0.452947,0.476525,0.357962
min,0.000000,164.000000,0.000000,149.000000,599.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,41473.000000,2.000000,86300.000000,171081.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,144769.000000,4.000000,113551.000000,171081.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,144769.000000,7.000000,113551.000000,171081.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,144769.000000,11.000000,113551.000000,171081.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,12.000000,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [101]:
%%time
evaluate_classifier(*train_test_split(standardize(processed_data), combined_target, test_size=0.5, random_state=123))

Accuracy of the model: 0.60239
Precision of the model: 0.6100223428024258
Recall of the model: 0.572192726837252
F1 score of the model: 0.5905022812239307
CPU times: user 4.99 s, sys: 1.33 s, total: 6.32 s
Wall time: 5.59 s


__Вывод:__